## 自定义输出解析器

在某些情况下，您可能希望实现自定义解析器以将模型输出构造为自定义格式。

有两种方法可以实现自定义解析器：
- 在 LCEL 中使用 RunnableLambda 或 RunnableGenerator – 我们强烈建议大多数用例使用此方法
- 通过从基类之一继承进行解析——这是困难方法

这两种方法之间的差异大多是表面的，主要在于触发哪些回调（例如， on_chain_start 与 on_parser_start ）

### 可运行的 Lambda 和生成器

推荐的解析方法是使用可运行的 lambda 和可运行的生成器！

在这里，我们将进行一个简单的解析，反转模型输出的大小写。

例如，如果模型输出：“Meow”，解析器将生成“mEOW”。


In [2]:
from typing import Iterable
from langchain_core.messages import AIMessage, AIMessageChunk

from langchain_openai import ChatOpenAI, OpenAI
API_KEY="sk-9846f14a2104490b960adbf5c5b3b32e" #老师的Key
BASE_URL = "https://api.deepseek.com"
MODEL= "deepseek-chat"

chat = ChatOpenAI(
    model= MODEL,  # 默认的大模型为GPT-3.5-turbo，比较便宜
    openai_api_base= BASE_URL,
    openai_api_key= API_KEY,
    temperature=0.3,
)


def parse(ai_message: AIMessage) -> str:
    """Parse the AI message."""
    return ai_message.content.swapcase()


chain = chat | parse
chain.invoke("Hello")

'hELLO! 😊 hOW CAN i ASSIST YOU TODAY?'

In [3]:
for chunk in chain.stream("tell me a shot story"):
    print(chunk, end="")

**tHE lAST lIGHT**  

iN A QUIET VILLAGE NESTLED BETWEEN MISTY MOUNTAINS, AN OLD LAMPLIGHTER NAMED eLIAS LIT THE STREETLIGHTS EVERY EVENING. tHE VILLAGERS RELIED ON HIS STEADY ROUTINE—UNTIL ONE NIGHT, THE LIGHTS WOULDN’T STAY LIT.  

wHISPERS SPREAD OF A SHADOW CREEPING THROUGH THE STREETS, EXTINGUISHING EACH FLAME. dETERMINED, eLIAS CARRIED HIS LAST LANTERN INTO THE HEART OF THE DARKNESS.  

"wHO ARE YOU?" HE CALLED.  

tHE SHADOW PAUSED. "i AM fORGOTTEN," IT MURMURED. "nO ONE REMEMBERS THE PAST, SO i TAKE THE LIGHT."  

eLIAS SMILED SADLY. "tHEN LET ME REMIND YOU." hE SPOKE OF FESTIVALS, LAUGHTER, AND LOVE—UNTIL THE SHADOW SHRANK, AND THE LANTERN BLAZED BRIGHTER THAN EVER.  

tHE NEXT MORNING, THE VILLAGE FOUND eLIAS GONE, BUT THE LIGHTS BURNED ON THEIR OWN. aND SOMETIMES, IN THE FLICKER, THEY SWORE THEY SAW HIS FACE.  

**tHE eND.**  

wOULD YOU LIKE A DIFFERENT THEME OR STYLE? 😊

In [4]:
from langchain_core.runnables import RunnableGenerator


def streaming_parse(chunks: Iterable[AIMessageChunk]) -> Iterable[str]:
    for chunk in chunks:
        yield chunk.content.swapcase()


streaming_parse = RunnableGenerator(streaming_parse)

In [5]:
chain = chat | streaming_parse
for chunk in chain.stream("tell me a story"):
    print(chunk, end="")

**tHE lOST sTAR**  

oNCE UPON A TIME, IN A SKY FILLED WITH COUNTLESS SHIMMERING STARS, THERE WAS A TINY STAR NAMED **lUNA**. uNLIKE THE OTHER STARS, lUNA WAS RESTLESS—SHE LONGED TO SEE THE WORLD BELOW, WHERE THE MOUNTAINS TOUCHED THE CLOUDS AND THE OCEANS SPARKLED UNDER THE MOONLIGHT.  

oNE NIGHT, WHEN THE WIND WAS STRONG, lUNA MADE A DARING DECISION. sHE LET GO OF THE SKY AND TUMBLED DOWN, LANDING SOFTLY IN A QUIET MEADOW. aT FIRST, SHE WAS THRILLED—FIREFLIES DANCED AROUND HER, AND THE SCENT OF WILDFLOWERS FILLED THE AIR. bUT SOON, SHE REALIZED SHE WAS LOST. wITHOUT HER PLACE IN THE SKY, SHE BEGAN TO DIM.  

a LITTLE GIRL NAMED **mIRA**, WHO LOVED STARGAZING, NOTICED THE FAINT GLOW IN THE GRASS. sHE CAREFULLY PICKED UP lUNA AND WHISPERED, *"yOU DON’T BELONG HERE, DO YOU?"*  

wITH A KIND HEART, mIRA CLIMBED THE TALLEST HILL, HELD lUNA UP HIGH, AND SANG A LULLABY HER GRANDMOTHER HAD TAUGHT HER. aS IF ANSWERING HER CALL, THE NIGHT SKY SHIMMERED, AND A GENTLE BREEZE LIFTED lUNA BACK IN